**DISCUSSION**:
* Do we just use CFTime everywhere?
* Do we try to infer calendars, or have them appended to the object or something?
* Should we switch to only importing the main objects for v2 so we can facilitate the object attribute?

**OTHER**:
* Exempt the S2S stuff from the nbstripout since the data takes so long to load.
* Couldn't test daily- or weekly-subx-example trying to pull data down. 

**TESTS**:
* Test that irregular initializations work (like MPI control with jumps in them).
* Test that normal series of dates work.
* Does the seasonal implementation work by just adding months?

**ISSUES**:
* Watch out that you're overwriting the `hind` object init for instance. Figure out how to not overwrite the object.
* Should transition to system where lead units is an attribute of the class. Need to think about how that would change based on coarsening, etc.
* Bootstrap perfect model also drops attrs.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr

from climpred import HindcastEnsemble, PerfectModelEnsemble
import climpred
from climpred.utils import *
from climpred.constants import *

import cftime
from climpred.tutorial import load_dataset

In [2]:
import xarray as xr
import numpy as np

In [ ]:
time1 = np.arange('1990', '2000', dtype='datetime64[Y]')
time2 = np.arange(len(time1))
ds1 = xr.DataArray(np.random.rand(len(time1)), dims=["time"], coords=[time1])
ds2 = xr.DataArray(np.random.rand(len(time2)), dims=["time"], coords=[time2])

In [3]:
def initialized_ds_lead0():
    da = load_dataset('CESM-DP-SST')
    da = da - da.mean('init')
    # Change to a lead-0 framework
    da['init'] += 1
    da['lead'] -= 1
    da['lead'].attrs['units'] = 'years'
    return da

def initialized_ds():
    da = load_dataset('CESM-DP-SST')
    da = da - da.mean('init')
    da['lead'].attrs['units'] = 'years'
    return da

def reconstruction_ds():
    da = load_dataset('FOSI-SST')
    da = da - da.mean('time')
    return da

In [7]:
ds1 = initialized_ds()
hind = initialized_ds_lead0()
reference = reconstruction_ds()

In [10]:
ds1["init"] = np.arange('1954', '2018', dtype='datetime64[Y]')

In [11]:
HindcastEnsemble(ds1).get_initialized()

<xarray.Dataset>
Dimensions:  (init: 64, lead: 10, member: 10)
Coordinates:
  * lead     (lead) int32 1 2 3 4 5 6 7 8 9 10
  * member   (member) int32 1 2 3 4 5 6 7 8 9 10
  * init     (init) object 1954-01-01 00:00:00 ... 2017-01-01 00:00:00
Data variables:
    SST      (init, lead, member) float64 -0.2354 -0.2171 ... 0.648 0.6407

In [16]:
ds1.init.to_index().shift(3, 'YS')

CFTimeIndex([1957-01-01 00:00:00, 1958-01-01 00:00:00, 1959-01-01 00:00:00,
             1960-01-01 00:00:00, 1961-01-01 00:00:00, 1962-01-01 00:00:00,
             1963-01-01 00:00:00, 1964-01-01 00:00:00, 1965-01-01 00:00:00,
             1966-01-01 00:00:00, 1967-01-01 00:00:00, 1968-01-01 00:00:00,
             1969-01-01 00:00:00, 1970-01-01 00:00:00, 1971-01-01 00:00:00,
             1972-01-01 00:00:00, 1973-01-01 00:00:00, 1974-01-01 00:00:00,
             1975-01-01 00:00:00, 1976-01-01 00:00:00, 1977-01-01 00:00:00,
             1978-01-01 00:00:00, 1979-01-01 00:00:00, 1980-01-01 00:00:00,
             1981-01-01 00:00:00, 1982-01-01 00:00:00, 1983-01-01 00:00:00,
             1984-01-01 00:00:00, 1985-01-01 00:00:00, 1986-01-01 00:00:00,
             1987-01-01 00:00:00, 1988-01-01 00:00:00, 1989-01-01 00:00:00,
             1990-01-01 00:00:00, 1991-01-01 00:00:00, 1992-01-01 00:00:00,
             1993-01-01 00:00:00, 1994-01-01 00:00:00, 1995-01-01 00:00:00,
            

In [ ]:
ds1['init'] = np.arange('1954', '2018', dtype='datetime64[Y]')

In [ ]:
ds1['init'] = np.arange(64)

In [ ]:
time_index

In [ ]:
isinstance(time_index, pd.DatetimeIndex)

In [ ]:
isinstance(ds1)

In [ ]:
not isinstance(ds1.init.to_index(), xr.CFTimeIndex)

In [ ]:
time_index = ds1['init'].to_index()

In [ ]:
isinstance(time_index, pd.Int64Index)

In [ ]:
isinstance(time_index, pd.Float64Index) | isinstance(
            time_index, pd.Int64Index)

In [ ]:
time_strings = [str(t) for t in time_index]

In [ ]:
split_dates = [d.split(' ')[0].split('-') for d in time_strings]

In [ ]:
cftime_dates = [
                cftime.DatetimeNoLeap(int(y), int(m), int(d))
                for (y, m, d) in split_dates
            ]

In [ ]:
time_index = xr.CFTimeIndex(cftime_dates)

In [ ]:
hindcast = HindcastEnsemble(ds1)

In [ ]:
hindcast.get_initialized().init

In [ ]:
hind = convert_time_index(hind, 'init', 'hind[init]')
reference = convert_time_index(reference, 'time', 'reference[time]')

In [ ]:
time_index = hind['init'].to_index()

In [ ]:
date = time_index[0]

In [ ]:
y,m,d = date.year, date.month, date.day

In [ ]:
y, m, d = time_index.year, time_index.month, time_index.day

In [ ]:
[cftime.DatetimeNoLeap(int(y), int(m), int(d)) for (y, m, d) in ]

In [ ]:
str(time_index[0]).split(' ')[0].split('-')

In [ ]:
time_strings = [str(t) for t in time_index]
split_dates = [d.split(' ')[0].split('-') for d in time_strings]

In [ ]:
cftime_dates = [
                cftime.DatetimeNoLeap(int(y), int(m), int(d))
                for (y, m, d) in split_dates
            ]



In [ ]:
metric='rmse'

In [ ]:
    # get metric function name, not the alias
    metric = METRIC_ALIASES.get(metric, metric)
    # get class metric(Metric)
    metric = get_metric_class(metric, DETERMINISTIC_HINDCAST_METRICS)
    if metric.probabilistic:
        raise ValueError(
            'probabilistic metric ', metric.name, 'cannot compute persistence forecast.'
        )

In [ ]:
offset_args_dict = get_lead_pdoffset_args(
                getattr(hind['lead'], 'units'), 1
            )

In [ ]:
offset_args_dict

In [ ]:
pd.DateOffset(**offset_args_dict)

In [ ]:
hind.init.dt.strftime('%Y%m%d 00:00') + pd.DateOffset(**offset_args_dict)

In [ ]:
hind.init.values + pd.DateOffset(**offset_args_dict)